# MAPPO Traffic Light Training - Automated Checkpoint Management

This notebook automates the entire training workflow on Kaggle:
- **Background training** with `nohup` (doesn't block notebook)
- **Automatic checkpoint packaging** (monitors and zips new checkpoints)
- **Dataset folder creation** (ready for Kaggle download)
- **Progress monitoring** (check logs anytime)

## Benefits
✅ Train for 9 hours uninterrupted  
✅ No manual zip creation needed  
✅ Checkpoints auto-packaged every hour  
✅ Download anytime from Output tab  

---

## 1️⃣ Install Dependencies & Setup

Install required packages and verify GPU availability.

In [ ]:
# Install SUMO
!apt-get update -qq
!apt-get install -y sumo sumo-tools sumo-doc > /dev/null 2>&1

# Set SUMO_HOME
import os
os.environ['SUMO_HOME'] = '/usr/share/sumo'

# Install Python packages
!pip install -q traci torch tensorboard

# Verify GPU
import torch
print(f"✅ PyTorch version: {torch.__version__}")
print(f"✅ CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"✅ GPU: {torch.cuda.get_device_name(0)}")
    print(f"✅ GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

## 2️⃣ Upload Code Files

Upload your training files to Kaggle. You need:
- `s1/mappo_k1_implementation.py` (Phase 1 improved version)
- `s1/k1.net.xml`
- `s1/k1_routes_3h_varying.rou.xml`
- `s1/k1_3h_varying.sumocfg`
- `kaggle_auto_checkpoint.py` (checkpoint manager)

**Option A:** Upload as Kaggle Dataset  
**Option B:** Use file upload widget below

In [ ]:
# Create s1 directory structure
!mkdir -p s1 mappo_models

# If files are in a Kaggle dataset, copy them
# Example: !cp -r /kaggle/input/traffic-light-code/* .

# Or use this to verify files exist
import os
required_files = [
    's1/mappo_k1_implementation.py',
    's1/k1.net.xml',
    's1/k1_routes_3h_varying.rou.xml',
    's1/k1_3h_varying.sumocfg',
    'kaggle_auto_checkpoint.py'
]

print("📁 Checking required files:")
for f in required_files:
    exists = os.path.exists(f)
    symbol = "✅" if exists else "❌"
    print(f"{symbol} {f}")

# If using file upload widget:
# from google.colab import files
# uploaded = files.upload()  # Upload kaggle_auto_checkpoint.py and mappo files

In [ ]:
# Verify subprocess module is available
import subprocess
import sys

print("✅ Subprocess module available")
print(f"✅ Python executable: {sys.executable}")
print(f"✅ Current directory: {os.getcwd()}")
print("\n📝 Ready for background training!")

## 3️⃣ Start Training in Background

Start training using Python subprocess (Jupyter-compatible). This allows:
- Training runs independently without blocking the notebook
- You can run other cells while training progresses
- Logs are saved to `training.log`
- Process continues until manually stopped or completes

**Note:** Unlike `nohup`, if the Kaggle kernel restarts, the process will stop. Save checkpoints frequently!

In [ ]:
# Start training in background using subprocess (Jupyter-compatible)
import subprocess
import time
import sys

# Start training process (non-blocking)
training_process = subprocess.Popen(
    [sys.executable, 's1/mappo_k1_implementation.py', '--num-episodes', '50', '--device', 'cuda'],
    stdout=open('training.log', 'w'),
    stderr=subprocess.STDOUT,
    cwd='.'
)

print(f"✅ Training started in background!")
print(f"   Process ID: {training_process.pid}")
print(f"   Episodes: 50")
print(f"   Device: cuda")

# Wait a moment to check if it started successfully
time.sleep(3)

# Check if process is still running
if training_process.poll() is None:
    print(f"\n✅ Training is running (PID: {training_process.pid})")
else:
    print(f"\n❌ Training process exited early with code: {training_process.poll()}")
    print("   Check training.log for errors")

print(f"\n📊 Monitor progress: Run the cells in section 5")
print(f"🛑 Stop training: Run cell in section 8")

## 4️⃣ Start Automatic Checkpoint Manager

Start the checkpoint monitor using subprocess. This:
- Checks for new checkpoints every hour (configurable)
- Automatically zips them into dataset folder
- Keeps only the last 3 checkpoints (saves space)
- Runs independently from training
- Logs to `checkpoint_monitor.log`

In [ ]:
# Start checkpoint monitor in background using subprocess
checkpoint_process = subprocess.Popen(
    [sys.executable, 'kaggle_auto_checkpoint.py', '--monitor', '--interval', '3600', '--keep-last', '3'],
    stdout=open('checkpoint_monitor.log', 'w'),
    stderr=subprocess.STDOUT,
    cwd='.'
)

time.sleep(2)

# Check if process is running
if checkpoint_process.poll() is None:
    print(f"✅ Checkpoint monitor started!")
    print(f"   Process ID: {checkpoint_process.pid}")
    print(f"   Check interval: 1 hour (3600s)")
    print(f"   Keeping: last 3 checkpoints")
    print(f"\n📦 Monitors: mappo_models/checkpoint_*")
    print(f"📂 Zips to: /kaggle/working/mappo-checkpoint-dataset/")
    print(f"📊 Check status: Run monitoring cells in section 5")
else:
    print(f"❌ Monitor process exited with code: {checkpoint_process.poll()}")
    print("   Check checkpoint_monitor.log for errors")

## 5️⃣ Monitor Training Progress

Run these cells anytime to check status. Training runs independently in background.

In [ ]:
# Check latest training logs (last 50 lines)
!tail -50 training.log

In [ ]:
# Check checkpoint monitor logs
!tail -20 checkpoint_monitor.log

In [ ]:
# Check running processes (using stored process objects)
print("🔍 Process Status:")
print("="*70)

# Check training process
if 'training_process' in globals():
    if training_process.poll() is None:
        print(f"✅ Training: RUNNING (PID: {training_process.pid})")
    else:
        print(f"❌ Training: STOPPED (exit code: {training_process.poll()})")
else:
    print("⚠️  Training: Not started yet (run cell 3)")

# Check checkpoint monitor
if 'checkpoint_process' in globals():
    if checkpoint_process.poll() is None:
        print(f"✅ Checkpoint monitor: RUNNING (PID: {checkpoint_process.pid})")
    else:
        print(f"❌ Checkpoint monitor: STOPPED (exit code: {checkpoint_process.poll()})")
else:
    print("⚠️  Checkpoint monitor: Not started yet (run cell 4)")

print("="*70)

In [ ]:
# List all checkpoints and their sizes
import os
import glob
from datetime import datetime

print("📦 Checkpoint Status:")
print("="*70)

# Original checkpoints
checkpoints = sorted(glob.glob("mappo_models/checkpoint_*"))
if checkpoints:
    print(f"\n🗂️  Raw Checkpoints (mappo_models/):")
    for ckpt in checkpoints[-5:]:  # Show last 5
        size = sum(os.path.getsize(os.path.join(ckpt, f)) 
                   for f in os.listdir(ckpt) if os.path.isfile(os.path.join(ckpt, f)))
        size_mb = size / (1024*1024)
        mod_time = datetime.fromtimestamp(os.path.getmtime(ckpt))
        print(f"  • {os.path.basename(ckpt)}: {size_mb:.1f} MB - {mod_time.strftime('%H:%M:%S')}")

# Zipped checkpoints
zips = sorted(glob.glob("/kaggle/working/mappo-checkpoint-dataset/*.zip"))
if zips:
    print(f"\n📦 Packaged Checkpoints (dataset folder):")
    total_size = 0
    for zip_path in zips:
        size_mb = os.path.getsize(zip_path) / (1024*1024)
        total_size += size_mb
        mod_time = datetime.fromtimestamp(os.path.getmtime(zip_path))
        print(f"  • {os.path.basename(zip_path)}: {size_mb:.1f} MB - {mod_time.strftime('%H:%M:%S')}")
    print(f"\n  Total size: {total_size:.1f} MB")
else:
    print("\n⏳ No checkpoints packaged yet (monitor runs every hour)")

print("="*70)

## 6️⃣ Manual Checkpoint Packaging (Optional)

If you want to package a specific checkpoint immediately instead of waiting for the hourly check:

In [ ]:
# Package latest checkpoint immediately
!python kaggle_auto_checkpoint.py

# Or package a specific checkpoint:
# !python kaggle_auto_checkpoint.py --checkpoint-dir mappo_models/checkpoint_time_20251124_120000

## 7️⃣ Download Checkpoints

Get download links for all packaged checkpoints. Click the links to download from Kaggle Output tab.

In [ ]:
from IPython.display import FileLink, display
import glob
import os

print("📥 Download Links for Packaged Checkpoints:")
print("="*70)

zips = sorted(glob.glob("/kaggle/working/mappo-checkpoint-dataset/*.zip"))

if zips:
    for zip_path in zips:
        size_mb = os.path.getsize(zip_path) / (1024*1024)
        print(f"\n📦 {os.path.basename(zip_path)} ({size_mb:.1f} MB)")
        display(FileLink(zip_path))
else:
    print("\n⏳ No checkpoint zips available yet")
    print("   The monitor packages checkpoints every hour")
    print("   Or run the manual packaging cell above")

print("\n" + "="*70)

## 8️⃣ Stop Training (When Done)

Stop training and checkpoint monitor when you're finished.

In [ ]:
# Stop training and checkpoint monitor processes
import time

print("🛑 Stopping processes...")

stopped = []
failed = []

# Stop training process
if 'training_process' in globals() and training_process.poll() is None:
    training_process.terminate()
    time.sleep(2)
    if training_process.poll() is None:
        training_process.kill()  # Force kill if needed
    stopped.append(f"Training (PID: {training_process.pid})")
else:
    failed.append("Training (already stopped or not started)")

# Stop checkpoint monitor
if 'checkpoint_process' in globals() and checkpoint_process.poll() is None:
    checkpoint_process.terminate()
    time.sleep(2)
    if checkpoint_process.poll() is None:
        checkpoint_process.kill()
    stopped.append(f"Checkpoint monitor (PID: {checkpoint_process.pid})")
else:
    failed.append("Checkpoint monitor (already stopped or not started)")

print("\n✅ Stopped:")
for item in stopped:
    print(f"   • {item}")

if failed:
    print("\n⚠️  Could not stop:")
    for item in failed:
        print(f"   • {item}")

print("\n📊 Final training log (last 20 lines):")
print("="*70)
try:
    with open('training.log', 'r') as f:
        lines = f.readlines()
        for line in lines[-20:]:
            print(line.rstrip())
except FileNotFoundError:
    print("No training log found")
print("="*70)

---

## 📋 Workflow Summary

### What Happens Automatically:

1. **Training runs in background** via `nohup`
   - Doesn't block notebook
   - Continues even if you close tab
   - Saves to `training.log`

2. **Checkpoint monitor runs independently**
   - Checks every hour for new checkpoints
   - Automatically zips complete checkpoints
   - Saves to `/kaggle/working/mappo-checkpoint-dataset/`
   - Keeps only last 3 to save space

3. **You can monitor anytime**
   - Check logs: `!tail training.log`
   - List checkpoints: Run cell 5
   - Download: Run cell 7

### No Manual Work Required! 🎉

**Before:** Save → Wait → Find checkpoint → Create zip → Move to dataset folder → Download

**Now:** Just monitor progress and download when ready!

---

## 🚀 Pro Tips

- **Resume training:** Add `--resume-checkpoint path/to/checkpoint` in cell 3
- **Change check interval:** Modify `--interval 3600` (default 1 hour)
- **Keep more checkpoints:** Change `--keep-last 3` to higher number
- **Quick package:** Run cell 6 for immediate packaging (don't wait for hourly check)

---